In [1]:
import cv2
import glob
import math

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

import numpy as np
import pandas as pd
import xgboost as xgb

from skimage.feature import hog
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

from moviepy.editor import VideoFileClip
from IPython.display import HTML

import sdc_project as SDC

In [2]:
clb = SDC.CameraCalibrator((1280, 720))
clb.load_images('camera_cal/*.jpg')
clb.match_points(9, 6)
clb.calibrate()

In [3]:
nonvehicles = pd.DataFrame(glob.glob('data/non-vehicles/*/*.png'), columns=['filename'])
nonvehicles = nonvehicles.sample(8000)
nonvehicles['auto'] = 0 

vehicles = pd.DataFrame(glob.glob('data/vehicles/*/*.png'), columns=['filename'])
vehicles = vehicles.sample(8000)
vehicles['auto'] = 1

data = nonvehicles.append(vehicles)
data = shuffle(data)

In [4]:
features = []
labels   = []
 
for i, row in data.iterrows():
    image = mpimg.imread(row['filename'])
    features.append(SDC.extract_features(image))
    labels.append(row['auto'])

print(len(features))
print(len(labels))

16000
16000


In [5]:
features = np.array(features)

selector = VarianceThreshold(0.0002)
selector.fit(features)
selected_features = selector.transform(features)

In [6]:
scaler = StandardScaler().fit(selected_features)
scaled_features = scaler.transform(selected_features)

In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(scaled_features, labels, test_size=0.33, random_state=42)

In [8]:
params = {
    'bst:max_depth': 2, 
    'bst:eta'      : 1, 
    'silent'       : 1, 
    'objective'    : 'binary:logistic',
    'nthread'      : 4,
    'eval_metric'  : 'auc'
}

num_round = 80

In [9]:
dtrain = xgb.DMatrix(X_train, y_train)
dvalid = xgb.DMatrix(X_valid, y_valid)

evallist  = [(dvalid, 'eval'), (dtrain, 'train')]

In [10]:
bst = xgb.train(params, dtrain, num_round, evallist)

[0]	eval-auc:0.986783	train-auc:0.991948
[1]	eval-auc:0.99007	train-auc:0.994293
[2]	eval-auc:0.993874	train-auc:0.997305
[3]	eval-auc:0.994863	train-auc:0.998203
[4]	eval-auc:0.996303	train-auc:0.998737
[5]	eval-auc:0.997126	train-auc:0.999358
[6]	eval-auc:0.997394	train-auc:0.999572
[7]	eval-auc:0.998126	train-auc:0.999829
[8]	eval-auc:0.998346	train-auc:0.999861
[9]	eval-auc:0.998845	train-auc:0.999954
[10]	eval-auc:0.998919	train-auc:0.999958
[11]	eval-auc:0.999066	train-auc:0.999976
[12]	eval-auc:0.999269	train-auc:0.999989
[13]	eval-auc:0.999355	train-auc:0.999994
[14]	eval-auc:0.999562	train-auc:0.999997
[15]	eval-auc:0.999615	train-auc:0.999999
[16]	eval-auc:0.999651	train-auc:0.999999
[17]	eval-auc:0.99968	train-auc:1
[18]	eval-auc:0.999698	train-auc:1
[19]	eval-auc:0.999713	train-auc:1
[20]	eval-auc:0.999735	train-auc:1
[21]	eval-auc:0.999745	train-auc:1
[22]	eval-auc:0.99975	train-auc:1
[23]	eval-auc:0.999782	train-auc:1
[24]	eval-auc:0.999798	train-auc:1
[25]	eval-auc:0.999

In [22]:
from imp import reload
reload(SDC)

<module 'sdc_project' from '/Users/Bombard/udacity/CarND-Vehicle-Detection/sdc_project.py'>

In [23]:
vd = SDC.VehicleDetector((720, 1280), selector, scaler, bst)

In [24]:
lf = SDC.LaneFinder((1280, 720), clb, vd)
clip   = VideoFileClip('project_video.mp4')
output = clip.fl_image(lf.process_image)
output.write_videofile('output_video/project_video.mp4', audio=False)

[MoviePy] >>>> Building video output_video/project_video.mp4
[MoviePy] Writing video output_video/project_video.mp4


100%|█████████▉| 1260/1261 [24:22<00:01,  1.10s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_video/project_video.mp4 

